# Ecuación de Onda

$$ \dfrac{\partial^2\phi}{\partial t^2} = v^2 \dfrac{\partial^2\phi}{\partial x^2}$$

Aproximando las derivadas se llega las siguientes ecuaciones
\begin{align}
\dfrac{d\phi}{dt} &= \psi_{(x,t)} \\
\dfrac{d\psi}{dt} &= \dfrac{v^2}{a^2}\left[\phi_{(x+a,t)} + \phi_{(x-a,t)} - 2\phi_{(x,t)} \right]
\end{align}

# Metodo explicito ( FTCS equations ) 

\begin{align}
\phi_{(x,t+h)} &= \phi_{(x,t)} + h\psi_{(x,t)} \\
\psi_{(x,t+h)} &= \psi_{(x,t)} + h \dfrac{v^2}{a^2}\left[\phi_{(x+a,t)} + \phi_{(x-a,t)} - 2\phi_{(x,t)}  \right]
\end{align}
Este metodo es numericamente inestable. Para remediar el problema de estabilidad se usa los "Metodos implicitos". 

# Metodo Implicito 

\begin{align}
\phi_{(x,t+h)} - h\psi_{(x,t+h)} &= \phi_{(x,t)}\\
\psi_{(x,t+h)} - h\dfrac{v^2}{a^2}\left[\phi_{(x+a,t+h)} + \phi_{(x-a,t+h)}- 2\phi_{(x,t+h)} \right] &= \psi_{(x,t)}
\end{align}

Este metodo es numericamente estable ( si se hace el analisis de estabilidad de Neumann). Segun el autor es incondicionalmente estable. 
- Sin embargo la estabilidad no es suficiente. Debido a que el analisis de neumann nos dice que el sistema decaera a una situación constante lo cual no es cierto para los problemas que describen la ecuación de Onda. Luego el Metodo implicito nos da soluciones estables pero que son NO fisicas. 

Entonces se necesita un metodo que este entre FTCS y el metodo implicito donde las soluciones ni decaen o crecen. Tal metodo es el metodo de Crank-Nicolson. 

# Metodo de Crank-Nicolson

\begin{align}
\phi_{(x,t+h)} - \dfrac{1}{2}h\psi_{(x,t+h)} &= \phi_{(x,t)} + \dfrac{1}{2}h\psi_{(x,t)} \\
\psi_{(x,t+h)} - h\dfrac{v^2}{2a^2}\left[ \phi_{(x+a,t+h)} + \phi_{(x-a,t+h)} - 2\phi_{(x,t+h)} \right] &= \psi_{(x,t)} + h\dfrac{v^2}{2a^2}\left[ \phi_{(x+a,t)} + \phi_{(x-a,t)} - 2\phi_{(x,t)} \right]
\end{align}

Estas ecuaciones son indirectas o mejor dicho implicitas. Debemos resolver un conjunto de ecuaciones simultaneas para dar los valores que queremos. 

<img src="img/933.jpg">

# Problema 9.8
# La ecuación de Schrodinger y el metodo de Crank-Nicolson method

$$ -\dfrac{h^2}{2M}\dfrac{\partial^2\psi}{\partial x^2} = i\hbar \dfrac{\partial \psi}{\partial t}$$

$$ \psi_{(x,t+h)} - h\dfrac{i \hbar}{4ma^2} \left[ \psi_{(x+a,t+h)} + \psi_{(x-a,t+h)} - 2 \psi_{(x,t+h)}  \right] = \psi_{(x,t)} + h\dfrac{i \hbar}{4ma^2} \left[ \psi_{(x+a,t)} + \psi_{(x-a,t)} - 2 \psi_{(x,t)} \right]$$ 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_banded

hbar = 6.582 * 10**(-16) # ev.s
ima = 1j
m = 0.510998928 *10**(6) * (1/(3*10**8))**2  # Mev/c^2

def crank_nicolson_schrodinguer(k=0,h=0,x1=0,gx1=0,x2=1,gx2=0,t0=0,gt0=0,tfinal=100):
    '''
    Esta funcion resuelve el problema de una dimension de Scrodinguer de la forma (-hbar^2)/(2*M) diff2x_u = i*hbar*difft_u
    Para eso te pide los siguientes argumentos 
    - k = cantidad paso temporal 
    - h = cantidad de paso espacial
    - x1 = extremo izquierdo espacial
    - gx1 = condicion de extremo izquierdo u(x=x1,t) = gx1
    - x2 = extremo derecho espacial
    - gx2 = condicion de extremo derecho  u(x=x2,t) = gx2
    - t0 = tiempo inicial de la simulacion
    - gt0 = es la condicion inicial , es decir u(x,t=t0) = gt0
    . tfinal = hasta que tiempo final se esta haciendo la simulacion 
    
    //// es bueno aclarar que h , k ,x1 ,x2 ,t0, tfinal deben ser coherente , es decir para que no ocurra cosas raras
    //// en hacer los pasos temporales y pasos espaciales 
    
    Esta funcion retorna un array donde cada columna representa el estado u(x,t=tiempo) en un tiempo especifico
    Es decir las filas son como a "x" como columna es a "tiempo"
    
    '''
    
    ############ Valores para calcular ###################
    a_1 = 1 + k*(ima*hbar)/(2*m*h**2) 
    a_2 = - k*(ima*hbar)/(4*m*h**2)
    b_1 = 1- k*(ima*hbar)/(2*m*h**2)
    b_2 = k*(ima*hbar)/(4*m*h**2)
    
    # calculamos cuantas iteraciones vamos a hacer para llegar al tiempo final
    pasos_temporal = int((tfinal-t0)/k)
    # calculamos el segemnto entre x1 y x2 en cuanto sera dividio
    pasos_espacial = int((x2-x1)/h)
    
    ########## creamos la matriz donde haremos todos los calculos 
    # el +1 hace la cuenta de la condicion inicial (OJO)
    u = np.zeros( shape=(pasos_espacial+1,pasos_temporal+1) , dtype = np.cdouble  ) 
    
    ####### LLenamos las condiciones de contorno
    u[0,0] = gx1(t0)
    # llenamos la condicion inicial temporal y espacial
    for i in range(1,pasos_espacial):
        x = x1 + i*h
        u[i,0] = gt0(x)
    for j in range(0,pasos_temporal+1):
        t = t0 + j*k
        u[0,j] = gx1(t)
        u[pasos_espacial,j] = gx2(t)
    
    ###### Comenzamos a hacer los pasos hacia adelante
    
    #### Creamos las matrices del problema
    d = [a_1]*(pasos_espacial-1) # el -1 se debe a que no se considera el extremo de paso espacial superior final
    d_u = [a_2]*(pasos_espacial-2) # el -2 por no ser la diagonal superior
    d_d = [a_2]*(pasos_espacial-2) # el -2 por no ser la diagonal inferior
    A = np.diag(d) + np.diag(d_u, 1) + np.diag(d_d, -1)
    
    d = [b_1]*(pasos_espacial-1) # el -1 se debe a que no se considera el extremo de paso espacial superior final
    d_u = [b_2]*(pasos_espacial-2)
    d_d = [b_2]*(pasos_espacial-2) 
    B = np.diag(d) + np.diag(d_u, 1) + np.diag(d_d, -1)
    
    for j in range(0,pasos_temporal):
        t = t0 + j*k
        for i in range(1,pasos_espacial):
            x = x1 + i*h
            
            ### resolvemos el problema matricial
            
            up = 1
            low = 1
            num = len(A)
            ab = np.zeros((low + up + 1, num),dtype = np.cdouble)

            for i1 in range(0,num):
                for j2 in range(0,num):
                    if (up+i1-j2 >= 0 and up+i1-j2 < low + up + 1):
                        ab[up + i1 - j2, j2] = A[i1,j2]
                        
            N = np.array( B.dot(u[1:pasos_espacial,j]), dtype = np.cdouble )
            # de la funcion scipy
            result = solve_banded((1, 1), ab, N) # resuelve el sistema 
            
            for l in range(1,pasos_espacial):
                u[l,j+1] = result[l-1]  # el -1 es debido a que result es un array de menor tamaño que la columna de j+1 de "u" 
             
    return u

In [2]:
# probemos nuestra funcion
def gx1prob(t):
    return 0
def gx2prob(t):
    return 0

L = 1e-8
N = 100
a = L/N
x_0 = L/2
sigma = 1e-10
k = 5e10

def gt0prob(x):
    if x == 0:
        return 0
    if x == L:
        return 0
    return np.exp(-(x-x_0)**2/(2*sigma**2))*np.exp(1j*k*x)

paso_tiempo = 10e-18
tiempo_final = 8000e-18
tiempo_inicial = 0
matriz = crank_nicolson_schrodinguer(k=paso_tiempo,h=a,x1=0,gx1=gx1prob,x2=L,gx2=gx2prob,
                      t0=tiempo_inicial,gt0=gt0prob,tfinal=tiempo_final)
display(matriz.shape)

matriz_real = matriz*np.conjugate(matriz)

(101, 801)

In [3]:
from ipywidgets import interactive

def plotting(i=0):
    foto_en_tiempo = matriz_real[::,i]
    #print(foto_en_tiempo)
    plt.figure(figsize=(9, 7))
    escalax = np.arange(0,L+a,a)
    ax = plt.plot(escalax,foto_en_tiempo)
    plt.title(f"Solución {0+ i*paso_tiempo} s")
    
interactive_plot = interactive(plotting,i=(0,int((tiempo_final-tiempo_inicial)/paso_tiempo),1))
interactive_plot

interactive(children=(IntSlider(value=0, description='i', max=800), Output()), _dom_classes=('widget-interact'…